In [1]:
# Constants
DF_PATH = "../data/processed/1_preprocessed_df.pkl"

NA_STRING = 'Not Specified'
TRANSPARENT_STRING = 'rgba(0, 0, 0)'

ROLE_COLS       = ['DevType']
TECH_COLS       = ['LanguageHaveWorkedWith',
                   'DatabaseHaveWorkedWith',
                   'PlatformHaveWorkedWith',
                   'WebframeHaveWorkedWith',
                   'MiscTechHaveWorkedWith',
                   'ToolsTechHaveWorkedWith',
                   'NEWCollabToolsHaveWorkedWith']
EXPORT_FEATURES_DIR = "../data/processed"

In [3]:
# Load packages
import pandas as pd
import numpy as np
import logging
import pickle
import os

import plotly
import plotly.express as px
import plotly.graph_objects as go # Graph objects are the low-level building blocks of figures which you can use instead of Plotly Express for greater control.

import matplotlib.cm as cm
import matplotlib.pyplot as plt

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE

from sklearn.pipeline import make_pipeline
from sklearn.cluster import AgglomerativeClustering
from sklearn.model_selection import train_test_split

pd.options.display.max_rows = 10000
pd.options.display.max_columns = 10000

### Read data and preprocess data

In [5]:
# Read Data 
df = pd.read_pickle(DF_PATH)

In [7]:
# One hot encode
encoded_dfs = {}
for col in ROLE_COLS + TECH_COLS :
    binarizer = MultiLabelBinarizer()
    encoded_df = pd.DataFrame(binarizer.fit_transform(df[col]),
                              columns = binarizer.classes_,
                              index= df[col].index)
    encoded_dfs[col] = encoded_df

# Merge 1-hot encoded
ohe_df = pd.concat(encoded_dfs, axis=1)

### Dimentionality reduction

In [ ]:
# Prepare sub frames
skills_ohe = ohe_df